In [0]:

from bdce.common.utils import update_application_code
update_application_code(spark, role="BDP-PROD-APP-INT-QA", application_name="tom-debug")
spark.sparkContext.addPyFile("/tmp/zeppelin_application_code/libs/python/dependencies.zip")



In [0]:


spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")


In [0]:

# Copyright (c) 2018 App Annie Inc. All rights reserved.
# pylint: disable=E1101,C0412,C1801

"""
DB Check modules
"""

import datetime

from applications.db_check_v1.cases.store.app_est_publisher_v1.constants import PUB_EST_GRANULARITY_DB_MAPPING, \
    PUB_EST_GRANULARITY_CHECK_RULES, PUB_EST_GRANULARITY_FREQ_MAPPING, CITUS_META_DSN, PUB_EST_DSN
from applications.db_check_v1.common.base_test import PipelineTest
from applications.db_check_v1.common.db_check_utils import query_df, etl_skip, query
from applications.db_check_v1.common.utils import get_date_list


DB_PUBLISHER_EST_METRICS = ["est_free_app_download", "est_paid_app_download", "est_revenue"]
PUB_EST_START_DATE = datetime.datetime(2010, 7, 4)


class PublisherEstPreAggrEst(PipelineTest):
    trigger_date_config = ('* 15 * * 2', 3)
    granularity = ""

    def check_aggr_by_sum_period(self, check_date, granularity):
        # start date of publisher data
        date_list = get_date_list(PUB_EST_START_DATE, check_date, freq=PUB_EST_GRANULARITY_FREQ_MAPPING[granularity])
        start_date = (datetime.datetime.strptime(date_list[-2], '%Y-%m-%d') + datetime.timedelta(days=1)) \
            .strftime('%Y-%m-%d')
        end_date = date_list[-1]
        sql_template = "SELECT sum(est_revenue) FROM store.{table}  where date between '{start}' and '{end}';"
        compare_table = PUB_EST_GRANULARITY_DB_MAPPING[PUB_EST_GRANULARITY_CHECK_RULES[granularity]]
        compare_sql = sql_template.format(table=compare_table, start=start_date, end=end_date)
        actual_table = PUB_EST_GRANULARITY_DB_MAPPING[granularity]
        actual_sql = sql_template.format(table=actual_table, start=start_date, end=end_date)

        compare_sum = query(PUB_EST_DSN, compare_sql)
        actual_sum = query(PUB_EST_DSN, actual_sql)
        msg = "actual_sum {} , compare_sum {}, actual_sql: {}, compare_sql: {}".format(
            actual_sum, compare_sum, actual_sql, compare_sql)
        self.assertEqual(compare_sum, actual_sum, msg)

    def check_meta_data_of_tables(self, granularity):
        sql_template = "select app_stats -> 'latest_available_date' as latest_available_date " \
                       "from meta where table_name = '{table_name}' and is_active = True ;"

        sql = sql_template.format(table_name=PUB_EST_GRANULARITY_DB_MAPPING[granularity].split("_v")[0])
        latest_available_date = query(CITUS_META_DSN, sql)[0][0]
        check_date = self._get_check_date_from_routing_config(datetime.datetime.now())
        date_list = get_date_list(PUB_EST_START_DATE, check_date, freq=PUB_EST_GRANULARITY_FREQ_MAPPING[granularity])
        self.assertEqual(latest_available_date, date_list[-1])

    @etl_skip()
    def test_publisher_est_pre_aggr_etl_completeness(self):
        self._testMethodName = 'test_publisher_est_{}_pre_aggr_etl_completeness'.format(self.granularity)
        self.check_aggr_by_sum_period(self.check_date, self.granularity)
        self.check_meta_data_of_tables(self.granularity)


class TestIOSPublisherWeeklyPreAggrEst(PublisherEstPreAggrEst):
    granularity = 'weekly'


class TestIOSPublisherMonthlyPreAggrEst(PublisherEstPreAggrEst):
    granularity = 'monthly'


class TestIOSPublisherQuarterlyPreAggrEst(PublisherEstPreAggrEst):
    granularity = 'quarterly'


class TestIOSPublisherYearlyPreAggrEst(PublisherEstPreAggrEst):
    granularity = 'yearly'



In [0]:


from applications.db_check_v1.common.base_test import debug
t_date = datetime.datetime(2017,01,04)
mycaseList = [
    TestIOSPublisherWeeklyPreAggrEst(trigger_datetime=t_date, methodName='test_publisher_est_pre_aggr_etl_completeness'),
    TestIOSPublisherMonthlyPreAggrEst(trigger_datetime=t_date, methodName='test_publisher_est_pre_aggr_etl_completeness'),
    TestIOSPublisherQuarterlyPreAggrEst(trigger_datetime=t_date, methodName='test_publisher_est_pre_aggr_etl_completeness'),
    TestIOSPublisherYearlyPreAggrEst(trigger_datetime=t_date, methodName='test_publisher_est_pre_aggr_etl_completeness')
]
#debug(caseList)


In [0]:

# Copyright (c) 2018 App Annie Inc. All rights reserved.
# pylint: disable=E1101,C0412,C1801

"""
DB Check modules
"""

import datetime

from applications.db_check_v1.common.base_test import PipelineTest
from applications.db_check_v1.common.db_check_utils import query_df, etl_skip
from applications.db_check_v1.common.constants import COUNTRY_CODE_MAPPING_BY_MARKET_CODE as COUNTRY_CODE_MAPPING, \
    CATEGORY_ID_MAPPING_BY_MARLKET_AND_DEVICE_CODE as CATEGORY_ID_MAPPING
from applications.db_check_v1.common.utils import get_week_start_end_date, get_date_list
from applications.db_check_v1.cases.usage.basic_kpi_v3.test_basic_kpi_v3_routine_plproxy import CITUS_DSN as PUB_EST_DSN
from applications.db_check_v1.cases.store.app_est_publisher_v1.constants import PUB_EST_DSN, PUB_EST_DB_METRICS


class PublisherEstRawData(object):
    raw_s3_path = "s3://b2c-prod-dca-store-estimates/store_estv2/PUB_ESTIMATES_FINAL/version=2.0.0/range_type=DAY" \
                  "/date={}/"
    device_code_mapping = {
        "00": "android-all",
        "01": "android-all",
        "02": "android-all",
        "10": "ios-phone",
        "11": "ios-phone",
        "12": "ios-phone",
        "1100": "ios-tablet",
        "1101": "ios-tablet",
        "1102": "ios-tablet",
        # "21000": "ios-all",
        # "21001": "ios-all",
        # "21002": "ios-all",
    }

    metric_mapping = {
        0: "free_app_download",
        1: "paid_app_download",
        2: "revenue",
        101: "free_app_download",
        100: "paid_app_download",
        102: "revenue",
        # 1000: "free_app_download",
        # 1001: "paid_app_download",
        # 1002: "revenue"
    }

    dimension_mapping = {
        "id": "publisher_id",
    }

    def __init__(self, spark):
        self.spark = spark

    def get(self, date, country_code):
        df = self._get_raw_data_by_date_country(date, country_code)
        df = self._parse_mapping(df)
        df = self._parse_unified_format(df)
        df = self._data_clean_up(df)
        return df

    def _data_clean_up(self, df):
        # clean unknown mapping
        category_id_list = list(set(CATEGORY_ID_MAPPING["apple-store"]["ios-all"].values() +
                                    CATEGORY_ID_MAPPING["google-play"]["android-all"].values()))

        country_code_list = list(set(COUNTRY_CODE_MAPPING["apple-store"].values() +
                                     COUNTRY_CODE_MAPPING["google-play"].values()))

        df = df[(df['category_id'].isin(category_id_list)) & (df['country_code'].isin(country_code_list))]
        return df

    def _parse_mapping(self, df):
        # country_code mapping
        df.loc[df["platform_id"] == 0] = df.loc[df["platform_id"] == 0].\
            replace({"store_id": COUNTRY_CODE_MAPPING["google-play"]})
        df.loc[df["platform_id"] == 1] = df.loc[df["platform_id"] == 1].\
            replace({"store_id": COUNTRY_CODE_MAPPING["apple-store"]})
        df = df.rename(columns={'store_id': 'country_code'})

        # category_id mapping
        df.loc[df["platform_id"] == 0] = df.loc[df["platform_id"] == 0].\
            replace({"category_id": CATEGORY_ID_MAPPING["google-play"]["android-all"]})
        df.loc[df["platform_id"] == 1] = df.loc[df["platform_id"] == 1].\
            replace({"category_id": CATEGORY_ID_MAPPING["apple-store"]["ios-all"]})

        # device_code mapping
        df["device_code"] = df["platform_id"].astype(str) + df["feed"].astype(str)
        df = df.replace({"device_code": self.device_code_mapping})

        # granularity
        df["granularity"] = "daily"

        # metrics mapping (from feed)
        df = df.replace({"feed": self.metric_mapping})
        return df

    def _parse_unified_format(self, df):
        df = df.rename(columns=self.dimension_mapping)
        df = df.pivot_table(index=["publisher_id", "category_id", "device_code", "country_code", "granularity"],
                            columns='feed', values='est')
        df.reset_index(inplace=True)
        df.columns.name = None
        return df

    def _get_raw_data_by_date_country(self, date, country_code):
        """
        +----------+--------+-----------+-----------+--------+----+----+-----+--------+
        |        id|store_id|category_id|platform_id|vertical|rank|feed|  est|platform|
        +----------+--------+-----------+-----------+--------+----+----+-----+--------+
        | 284417353|       0|       6006|          1|       1|   1|1002|45235|     ios|
        | 349554266|       0|       6006|          1|       1|   2|1002|20732|     ios|
        |1316153435|       0|       6006|          1|       1|   3|1002|15136|     ios|
        +----------+--------+-----------+-----------+--------+----+----+-----+--------+
        """
        ios_store_ids = [str(k) for k, v in COUNTRY_CODE_MAPPING["apple-store"].items() if v == country_code]
        gp_store_ids = [str(k) for k, v in COUNTRY_CODE_MAPPING["google-play"].items() if v == country_code]
        raw_df = self.spark.read.parquet(self.raw_s3_path.format(date)).\
            filter('store_id in ({})'.format(",".join(ios_store_ids + gp_store_ids))).toPandas()
        return raw_df

    def get_metrics_count(self, date):
        ios_store_id = [str(s_id) for s_id in COUNTRY_CODE_MAPPING["apple-store"].keys()]
        gp_store_id = [str(s_id) for s_id in COUNTRY_CODE_MAPPING["google-play"].keys()]
        # ios_category_id = [str(c_id) for c_id in CATEGORY_ID_MAPPING["apple-store"]["ios-all"].keys()]
        # gp_category_id = [str(c_id) for c_id in CATEGORY_ID_MAPPING["google-play"]["android-all"].keys()]

        fillter_sql = "platform_id = {} and store_id in ({}) and feed in ({})"
        df = self.spark.read.parquet(self.raw_s3_path.format(date))
        feed_ids_sql = ",".join([str(x) for x in self.metric_mapping.keys()])

        count_all = df.filter(fillter_sql.format(1, ",".join(ios_store_id), feed_ids_sql)).count() + \
                    df.filter(fillter_sql.format(0, ",".join(gp_store_id), feed_ids_sql)).count()
        return count_all

    def get_v1_raw_metrics_count(self, date):
        ios_store_id = [str(s_id) for s_id in COUNTRY_CODE_MAPPING["apple-store"].keys()]
        gp_store_id = [str(s_id) for s_id in COUNTRY_CODE_MAPPING["google-play"].keys()]

        df = self.spark.read.option("delimiter", "\t").csv(
            "s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/{}/*/sbe_est_publisher/*/*.csv.gz".format(date))
        fillter_sql = "_c2 = {} and _c0 in ({})"  # _c2 > platform_id, _c0 > store_id

        count_all = df.filter(fillter_sql.format(1, ",".join(ios_store_id))).count() + \
                    df.filter(fillter_sql.format(0, ",".join(gp_store_id))).count()
        return count_all


class PublisherEstUnifiedData(object):
    unified_s3_path = "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-publisher-dna-log.v1/" \
                      "fact/granularity=daily/date={}/"
    available_device_codes = ['ios-phone', 'ios-tablet', 'android-all']

    def __init__(self, spark):
        self.spark = spark

    def get(self, date, country_code):
        unified_df = self.spark.read.format("delta").\
            load(self.unified_s3_path.format(date)).filter("country_code = '{}'".format(country_code)).toPandas()
        unified_df = unified_df.drop(["_identifier", "revenue_iap", "revenue_non_iap", "date"], axis=1)
        return unified_df

    def get_metrics_count(self, date):
        df = self.spark.read.format("delta").load(self.unified_s3_path.format(date))
        metrics_count = 0
        device_code_list_sql = "','".join(self.available_device_codes)
        for metric in PUB_EST_DB_METRICS:
            metrics_count += df.filter("device_code in ('{}') and {} is not null".format(device_code_list_sql,
                                                                                       metric)).count()
        return metrics_count


class PublisherEstDBData(object):
    def get(self, date):
        sql = "SELECT * FROM store.store_est_publisher_fact_v2 WHERE date='{}'".format(date)
        return query_df(PUB_EST_DSN, sql)

    def get_metrics_count(self, date):
        metrics_count = 0
        for metric in PUB_EST_DB_METRICS:
            sql = "SELECT count(*) AS metrics_count FROM store.store_est_publisher_fact_v2 WHERE date='{}' AND {} IS NOT NULL".format(date, metric)
            data = query_df(PUB_EST_DSN, sql)
            metrics_count += data.loc[0].metrics_count
        return metrics_count


class TestPublisherEstWeekly(PipelineTest):
    # Every Tuesday 15:00 UTC(23:00 BJ) time will refresh the data of last Full Week.
    trigger_date_config = ('* 15 * * 2', 3)

    def _compare_df(self, df1, df2, log=''):
        for diff_type in ["left", "right"]:
            diff_df = df1.merge(df2, indicator=True, how=diff_type)  # .loc[lambda x : x['_merge']!='both']
            diff_df = diff_df.loc[diff_df["_merge"] != "both"]
            if len(diff_df) != 0:
                print diff_type
                print "dataframe overview of df1 and df2"
                print df1
                print df2
                print "dimension overview of diff df"
                print diff_df.country_code.unique()
                print diff_df.category_id.unique()
                print diff_df.device_code.unique()
            self.assertEqual(len(diff_df), 0,
                             msg="found mismatch when compare the raw, unified, db."
                                 " diff count is \n {}, logs:{}".format(len(diff_df), log))

    # @etl_skip()
    # def test_publisher_est_etl_accuracy(self):
    #     # Every Tuesday 16:00 UTC time will refresh the data of last Full Week.
    #     country_code = 'US'
    #     start_date, end_date = get_week_start_end_date(self.check_date_str)
    #     date_list = get_date_list(start_date, end_date)
    #     for date in date_list:
    #         raw_df = PublisherEstRawData(self.spark).get(date, country_code)
    #         unified_df = PublisherEstUnifiedData(self.spark).get(date, country_code)
    #         db_df = PublisherEstDBData().get(date)
    #
    #         self._compare_df(raw_df, unified_df, log="raw / unified - {}".format(date))
    #         self._compare_df(unified_df, db_df, log="unified / db - {}".format(date))

    @etl_skip()
    def test_publisher_est_etl_completeness(self):
        start_date, end_date = get_week_start_end_date(self.check_date_str)
        date_list = get_date_list(start_date, end_date)
        for date in date_list:
            raw_count = PublisherEstRawData(self.spark).get_metrics_count(date)
            unified_count = PublisherEstUnifiedData(self.spark).get_metrics_count(date)
            db_count = PublisherEstDBData().get_metrics_count(date)
            self.assertEqual(raw_count, unified_count)
            self.assertEqual(raw_count, db_count)
            self.assertTrue(db_count>0)

    def test_publisher_est_etl_timelines(self):
        # Every Tuesday 15:00 UTC(23:00 BJ) time will refresh the data of last Full Week.
        # E.g. 2020-02-12 10:00 the data of 2020-02-02 ~ 2020-02-08 will be ready
        trigger_datetime = datetime.datetime.strptime("2020-02-12 9:00:00", '%Y-%m-%d %H:%M:%S')
        check_date_str_actual = self._get_check_date_from_routing_config(trigger_datetime).strftime("%Y-%m-%d")
        self.assertEqual("2020-02-08", check_date_str_actual)

t_date = datetime.datetime(2020,8,5)


mycaselist2 = [
    TestPublisherEstWeekly(trigger_datetime=t_date, methodName='test_publisher_est_etl_timelines')
]




In [0]:

print t_date
print 123
debug(mycaselist2)


In [0]:




from aadatapipelinecore.core.utils import email

def send_db_check_email(title, text_context, key=None):
    default_recipients = ["tom@appannie.com"]
    if 'Failed' in title or (key is not None and 'Failed' in key):
        default_recipients.append("tom@appannie.com")
    email.send(title, text_context, default_recipients, sender='dev-qa-data-quality@appannie.com')



import datetime
import unittest

from aadatapipelinecore.core.monitoring.pipeline_monitor import running, fail, task_success
from aadatapipelinecore.core.urn import Urn
from aadatapipelinecore.core.utils.identifier import package_id
from aadatapipelinecore.core.utils.spark import canned_spark, stop

from applications.db_check_v1.cases.store.app_est_publisher_v1.test_app_est_publisher_aggr_v1 import \
    TestPublisherWeeklyPreAggrEst, TestPublisherMonthlyPreAggrEst, TestPublisherQuarterlyPreAggrEst, \
    TestPublisherYearlyPreAggrEst
from applications.db_check_v1.cases.store.app_est_publisher_v1.test_app_est_publisher_v1 import \
    TestPublisherEstPreviewDaily, TestPublisherEstFinalWeekly
from applications.db_check_v1.cases.store.download_attribution.test_download_attribution_pre_agg import \
    TestStoreDownloadAttrPreAgg

from applications.db_check_v1.cases.store.genre_est_v1.test_game_iq_market_size_full_restatement import \
    TestGameIQMarketSizeFullRestatement
from applications.db_check_v1.cases.advanced_review.test_advainced_review import TestAdvancedReview
from applications.db_check_v1.cases.store.store_download_revenue.store_daily import TestStoreDownloadRevenue
from applications.db_check_v1.cases.store.store_download_revenue.store_meta_db import TestStoreMetaDB
from applications.db_check_v1.cases.store.store_download_revenue.store_pre_agg import \
    TestStoreDownloadRevenuePreAgg
from applications.db_check_v1.cases.usage.genre_cross_v1.test_game_iq_cross_genre import TestGameIQCrossGenre
from applications.db_check_v1.cases.advanced_review.test_topic_join_review import TestTopicJoinReview
from applications.db_check_v1.cases.aso.test_aso import TestASO, TestASOSOVDataDaily, TestASOMetrics
from applications.db_check_v1.cases.market.test_market import TestMarketDimensionDaily, \
    TestMarketLogsFactAndSeenDaily, TestMarketMonthlyCheck, TestMarketWeeklyCheck
from applications.db_check_v1.cases.mobile_web.test_mobile_web import TestMobileWebDaily, \
    TestMobileWebWeekly, \
    TestMobileWebMonthly, TestMobileWebRetention, TestMobilWebDomainMeta
from applications.db_check_v1.cases.sdk.test_sdk import TestSDKDaily, TestSDKWeekly, TestSDKMonthly
from applications.db_check_v1.cases.store.app_rank_v1.test_app_rank_v1 import TestAppStoreRankDaily
from applications.db_check_v1.cases.store.app_rank_v1.test_app_store_rank import TestAppStoreRank
from applications.db_check_v1.cases.store.download_attribution.test_download_attribution import \
    TestStoreDownloadAttributionDB, TestStoreDownloadAttributionUnified, TestDownloadAttribution
from applications.db_check_v1.cases.store.market_size_v1.test_market_size_v1 import TestMarketSizeWeekly
from applications.db_check_v1.cases.usage.test_city_level import TestCityLevelWeekly, TestCityLevelMonthly
from applications.db_check_v1.cases.store.store_app_v1.test_app_detail import TestAppDetailCompleteness
from applications.db_check_v1.cases.usage.basic_kpi_v3.test_basic_kpi_v3_routine_raw import \
    TestUsageRoutineRawCompletenessPreviewWeekly, TestUsageRoutineRawCompletenessPreviewMonthly, \
    TestUsageRoutineRawCompletenessPreviewDaily, TestUsageRoutineRawAccuracyMonthly, \
    TestUsageRoutineRawAccuracyWeekly, TestUsageRoutineRawAccuracyDaily, TestUsageRoutineRawCompletenessFinalDaily, \
    TestUsageRoutineRawCompletenessFinalWeekly, TestUsageRoutineRawCompletenessFinalMonthly
from applications.db_check_v1.cases.usage.basic_kpi_v3.test_basic_kpi_v3_routine_plproxy import \
    TestUsageRoutinePlproxyCompletenessPreviewDaily, TestUsageRoutinePlproxyCompletenessPreviewWeekly, \
    TestUsageRoutinePlproxyCompletenessPreviewMonthly, TestUsageRoutinePlproxyCompletenessFinalDaily, \
    TestUsageRoutinePlproxyCompletenessFinalMonthly, TestUsageRoutinePlproxyCompletenessFinalWeekly
from applications.db_check_v1.common.html_report_test_runner import HTMLTestRunner

suite = unittest.TestSuite()

def send_message():
    log_file = "/tmp/db_check.log"
    with open(log_file, "w") as html_file:
        
        suite.addTests(unittest.TestLoader().loadTestsFromTestCase(TestDownloadAttribution))
        suite.addTests(unittest.TestLoader().loadTestsFromTestCase(TestASO))
        suite.addTests(unittest.TestLoader().loadTestsFromTestCase(TestASOSOVDataDaily))
        suite.addTests(unittest.TestLoader().loadTestsFromTestCase(TestASOMetrics))
        suite.addTests(unittest.TestLoader().loadTestsFromTestCase(TestMobileWebDaily))
        suite.addTests(unittest.TestLoader().loadTestsFromTestCase(TestMobileWebWeekly))
        suite.addTests(unittest.TestLoader().loadTestsFromTestCase(TestMobileWebMonthly))
        suite.addTests(unittest.TestLoader().loadTestsFromTestCase(TestMobileWebRetention))
        suite.addTests(unittest.TestLoader().loadTestsFromTestCase(TestMobilWebDomainMeta))
        suite.addTests(unittest.TestLoader().loadTestsFromTestCase(TestCityLevelWeekly))
        suite.addTests(unittest.TestLoader().loadTestsFromTestCase(TestCityLevelMonthly))
        suite.addTests(unittest.TestLoader().loadTestsFromTestCase(TestSDKDaily))
        suite.addTests(unittest.TestLoader().loadTestsFromTestCase(TestSDKWeekly))
        suite.addTests(unittest.TestLoader().loadTestsFromTestCase(TestSDKMonthly))
        suite.addTests(unittest.TestLoader().loadTestsFromTestCase(TestAdvancedReview))
        suite.addTests(unittest.TestLoader().loadTestsFromTestCase(TestMarketDimensionDaily))
        suite.addTests(unittest.TestLoader().loadTestsFromTestCase(TestMarketLogsFactAndSeenDaily))
        suite.addTests(unittest.TestLoader().loadTestsFromTestCase(TestMarketWeeklyCheck))
        suite.addTests(unittest.TestLoader().loadTestsFromTestCase(TestMarketMonthlyCheck))
        suite.addTests(unittest.TestLoader().loadTestsFromTestCase(TestAppStoreRank))
        suite.addTests(unittest.TestLoader().loadTestsFromTestCase(TestAppStoreRankDaily))
        suite.addTests(unittest.TestLoader().loadTestsFromTestCase(TestMarketSizeWeekly))
        suite.addTests(unittest.TestLoader().loadTestsFromTestCase(TestTopicJoinReview))
        suite.addTests(unittest.TestLoader().loadTestsFromTestCase(TestGameIQCrossGenre))
        suite.addTests(unittest.TestLoader().loadTestsFromTestCase(TestGameIQMarketSizeFullRestatement))
        suite.addTests(unittest.TestLoader().loadTestsFromTestCase(TestAppDetailCompleteness))
        suite.addTests(unittest.TestLoader().loadTestsFromTestCase(TestUsageRoutineRawCompletenessPreviewDaily))
        suite.addTests(unittest.TestLoader().loadTestsFromTestCase(TestUsageRoutineRawCompletenessPreviewWeekly))
        suite.addTests(unittest.TestLoader().loadTestsFromTestCase(TestUsageRoutineRawCompletenessPreviewMonthly))
        suite.addTests(unittest.TestLoader().loadTestsFromTestCase(TestUsageRoutineRawAccuracyDaily))
        suite.addTests(unittest.TestLoader().loadTestsFromTestCase(TestUsageRoutineRawAccuracyWeekly))
        suite.addTests(unittest.TestLoader().loadTestsFromTestCase(TestUsageRoutineRawAccuracyMonthly))
        suite.addTests(unittest.TestLoader().loadTestsFromTestCase(TestUsageRoutinePlproxyCompletenessPreviewDaily))
        suite.addTests(unittest.TestLoader().loadTestsFromTestCase(TestUsageRoutinePlproxyCompletenessPreviewWeekly))
        suite.addTests(unittest.TestLoader().loadTestsFromTestCase(TestUsageRoutinePlproxyCompletenessPreviewMonthly))
        suite.addTests(unittest.TestLoader().loadTestsFromTestCase(TestUsageRoutineRawCompletenessFinalDaily))
        suite.addTests(unittest.TestLoader().loadTestsFromTestCase(TestUsageRoutineRawCompletenessFinalWeekly))
        suite.addTests(unittest.TestLoader().loadTestsFromTestCase(TestUsageRoutineRawCompletenessFinalMonthly))
        suite.addTests(unittest.TestLoader().loadTestsFromTestCase(TestUsageRoutinePlproxyCompletenessFinalDaily))
        suite.addTests(unittest.TestLoader().loadTestsFromTestCase(TestUsageRoutinePlproxyCompletenessFinalWeekly))
        suite.addTests(unittest.TestLoader().loadTestsFromTestCase(TestUsageRoutinePlproxyCompletenessFinalMonthly))
        suite.addTests(unittest.TestLoader().loadTestsFromTestCase(TestStoreDownloadRevenue))
        suite.addTests(unittest.TestLoader().loadTestsFromTestCase(TestStoreDownloadRevenuePreAgg))
        suite.addTests(unittest.TestLoader().loadTestsFromTestCase(TestStoreDownloadAttrPreAgg))
        suite.addTests(unittest.TestLoader().loadTestsFromTestCase(TestStoreMetaDB))
        suite.addTests(unittest.TestLoader().loadTestsFromTestCase(TestStoreDownloadAttributionDB))
        suite.addTests(unittest.TestLoader().loadTestsFromTestCase(TestStoreDownloadAttributionUnified))
        suite.addTests(unittest.TestLoader().loadTestsFromTestCase(TestPublisherEstPreviewDaily))
        suite.addTests(unittest.TestLoader().loadTestsFromTestCase(TestPublisherEstFinalWeekly))
        suite.addTests(unittest.TestLoader().loadTestsFromTestCase(TestPublisherWeeklyPreAggrEst))
        suite.addTests(unittest.TestLoader().loadTestsFromTestCase(TestPublisherMonthlyPreAggrEst))
        suite.addTests(unittest.TestLoader().loadTestsFromTestCase(TestPublisherQuarterlyPreAggrEst))
        suite.addTests(unittest.TestLoader().loadTestsFromTestCase(TestPublisherYearlyPreAggrEst))
        # runner = unittest.TextTestRunner(file, verbosity=2, buffer=True)

        runner = HTMLTestRunner(
            stream=html_file,
            title='DB Test Report',
            description='This db_check the report output by Tech Team.'
        )

        failed_count = 0
        result_list = runner.run(suite).result
        for result in result_list:
            if result[0] == 1 or result[0] == 2:
                failed_count += 1

    with open(log_file, 'r') as html_file:
        today = datetime.date.today()
        str_today = today.strftime("%Y-%m-%d")

        title = "Data Refresh Check Report - " + str_today + " - "
        if failed_count == 0:
            title += "Passed"
        else:
            title += "Failed"
        send_db_check_email(title, html_file.read())


send_message()